In [1]:
from datasets import load_dataset

dataset_name = "dim/nfs_pix2pix_1920_1080_v5"
# dataset_name = "dim/nfs_pix2pix_1920_1080_v5_upscale_2x_raw"
# dataset_name = "dim/nfs_pix2pix_1920_1080_v6"
# dataset_name = "dim/render_nfs_4screens_6_sdxl_1_wan_mix"
# dataset_name = "dim/render_nfs_4screens_5_sdxl_1_wan_mix"
dataset = load_dataset(
    dataset_name,
    cache_dir=f"/code/dataset/{dataset_name.split('/')[-1]}",
)
# dataset["train"] = dataset["train"].shuffle(seed=2025)
dataset = dataset["train"]

/code/auto_remaster/sandbox/DiffSynth-Studio/.venv_diff/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 3. Подготовка трансформаций
from torchvision import transforms
import torch

weight_dtype = torch.bfloat16
# weight_dtype = torch.float32
device = "cuda"
resolution = 512
valid_transforms = transforms.Compose(
    [
        transforms.Resize(
            resolution,
            interpolation=transforms.InterpolationMode.LANCZOS,
        ),
        transforms.CenterCrop(resolution),
    ]
)
train_transforms = transforms.Compose(
    [
        transforms.Resize(
            resolution,
            interpolation=transforms.InterpolationMode.LANCZOS,
        ),
        transforms.CenterCrop(resolution),
        transforms.ToTensor(),
        transforms.Normalize(
            (0.5, 0.5, 0.5),
            (0.5, 0.5, 0.5),
        ),
    ]
)

In [3]:
pos = 20
item = dataset[pos]
source_image_name = "input_image"
target_image_name = "edited_image"
# Подготовка исходных изображений для визуализации и метрик
orig_source_pil = item[source_image_name].convert("RGB")
target_pil = item[target_image_name].convert("RGB")

orig_source_pil = item[source_image_name].convert("RGB")
target_pil = item[target_image_name].convert("RGB")

source_tensor = valid_transforms(orig_source_pil)
target_tensor = valid_transforms(target_pil)

c_t = train_transforms(orig_source_pil).unsqueeze(0).cuda()

In [4]:
c_t.shape

torch.Size([1, 3, 512, 512])

In [5]:
import os, sys

os.environ["DIFFSYNTH_MODEL_BASE_PATH"] = (
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/"
)
from diffsynth.pipelines.flux2_image import Flux2ImagePipeline, ModelConfig
import torch

vram_config = {
    # "offload_dtype": "disk",
    # "offload_device": "disk",
    "onload_dtype": torch.float8_e4m3fn,
    # "onload_device": "cpu",
    "preparing_dtype": torch.float8_e4m3fn,
    "preparing_device": "cuda",
    "computation_dtype": torch.bfloat16,
    "computation_device": "cuda",
}
checkpoint_path = "auto_remaster/sandbox/DiffSynth-Studio/models/train/FLUX.2-klein-9B_lora/epoch-0.safetensors"
pipe = Flux2ImagePipeline.from_pretrained(
    torch_dtype=torch.bfloat16,
    # torch_dtype=torch.float8_e4m3fn,
    device="cuda",
    model_configs=[
        ModelConfig(
            model_id="black-forest-labs/FLUX.2-klein-9B",
            origin_file_pattern="text_encoder/*.safetensors",
            computation_dtype=torch.float8_e4m3fn,
            **{
                # "offload_dtype": "disk",
                # "offload_device": "disk",
                "onload_dtype": torch.float8_e4m3fn,
                # "onload_device": "cpu",
                "preparing_dtype": torch.float8_e4m3fn,
                "preparing_device": "cuda",
                # "computation_dtype": torch.bfloat16,
                "computation_device": "cuda",
            },
        ),
        ModelConfig(
            model_id="black-forest-labs/FLUX.2-klein-9B",
            origin_file_pattern="transformer/*.safetensors",
            **vram_config,
        ),
        ModelConfig(
            model_id="black-forest-labs/FLUX.2-klein-9B",
            origin_file_pattern="vae/diffusion_pytorch_model.safetensors",
            **vram_config,
        ),
    ],
    tokenizer_config=ModelConfig(
        model_id="black-forest-labs/FLUX.2-klein-9B", origin_file_pattern="tokenizer/"
    ),
)
pipe.load_lora(pipe.dit, checkpoint_path)

2026-01-30 23:56:10,617 - modelscope - INFO - Target directory already exists, skipping creation.


Loading models from: [
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/text_encoder/model-00003-of-00004.safetensors",
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/text_encoder/model-00002-of-00004.safetensors",
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/text_encoder/model-00001-of-00004.safetensors",
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/text_encoder/model-00004-of-00004.safetensors"
]
Loaded model: {
    "model_name": "z_image_text_encoder",
    "model_class": "diffsynth.models.z_image_text_encoder.ZImageTextEncoder",
    "extra_kwargs": {
        "model_size": "8B"
    }
}


2026-01-30 23:56:14,880 - modelscope - INFO - Target directory already exists, skipping creation.


Loading models from: [
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/transformer/diffusion_pytorch_model-00002-of-00002.safetensors",
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/transformer/diffusion_pytorch_model-00001-of-00002.safetensors"
]
Loaded model: {
    "model_name": "flux2_dit",
    "model_class": "diffsynth.models.flux2_dit.Flux2DiT",
    "extra_kwargs": {
        "guidance_embeds": false,
        "joint_attention_dim": 12288,
        "num_attention_heads": 32,
        "num_layers": 8,
        "num_single_layers": 24
    }
}


2026-01-30 23:56:19,166 - modelscope - INFO - Target directory already exists, skipping creation.


Loading models from: "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/vae/diffusion_pytorch_model.safetensors"
Loaded model: {
    "model_name": "flux2_vae",
    "model_class": "diffsynth.models.flux2_vae.Flux2VAE",
    "extra_kwargs": null
}
No flux2_text_encoder models available. This is not an error.
Using z_image_text_encoder from [
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/text_encoder/model-00003-of-00004.safetensors",
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/text_encoder/model-00002-of-00004.safetensors",
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/text_encoder/model-00001-of-00004.safetensors",
    "/code/auto_remaster/sandbox/DiffSynth-Studio/models/black-forest-labs/FLUX.2-klein-9B/text_encoder/model-00004-of-00004.safetensors"
].
Using flux2_dit from [
    "/code/auto_remaster/sandbox/DiffSynth

2026-01-30 23:56:21,957 - modelscope - INFO - Target directory already exists, skipping creation.


144 tensors are fused by LoRA. Fused LoRA layers cannot be cleared by `pipe.clear_lora()`.


In [6]:
prompt = "make this image photorealistic"
image = pipe(
    prompt=prompt,
    seed=0,
    num_inference_steps=40,
    cfg_scale=4,
    height=512,
    width=512,
    # edit_image=
    input_image=c_t,
)
# image.save("image.jpg")

NotImplementedError: "mul_cuda" not implemented for 'Float8_e4m3fn'